Processing Database

In [4]:
import src.notion_API as notion
import src.pocketbase_API as pocket
from typing import  Callable

In [5]:
def get_all(getFunction: Callable[[dict, bool], any], params: dict):

    all_params = {'perPage': 500}
    all_params.update(params)
    first = getFunction(all_params, full=True).json()
    items = first['items']
    for page in range(1,first['totalPages']):
        all_params.update({'page': (page+1)})
        output = getFunction(all_params, full=True).json()
        items = items + output['items']

    return items

In [6]:
def update_action_counts():

    # Run Times
    #   3m 25.9s (first run, no improvements)
    #   3m 13.5s (second rum, no improvements)
    #   3m 3.9s (first run, added #2)
    #   2m 59.9s (second run ,added #2)
    #   3m 28.9s (first run, added #1,#2)

    # Lets run some performance tests

    # Do we need to
    #   1. not use a for-loop -> map?(No), list comprehension(slower)
    #   2. use Python's filter function to get counts?
    def count_iterable(i):
        return sum(1 for e in i)
    #   3. Batch the updates
    #   4. check when the last update was (ID of what day) and update counts on days greater than last update
    
    # Get all actions (This will need to be changed as run time is 3m 25.9s)
    actions = get_all(pocket.get_actions,{})
    # Get all days
    days = get_all(pocket.get_days,{})

    # [pocket.patch_action(id=action['id'],json={"total_count": count_iterable(filter(lambda y: y['action'] == action['id'], days))}) for action in actions]

    # Loop through all actions and filter days by their action IDs. (Filter in Python?)
    for action in actions:
        id = action['id']
        name = action['action']
        # action_count = len(get_all(pocket.get_days,{'filter': f'action="{id}"'}))
        action_count = count_iterable(filter(lambda x: x['action'] == id, days))
        pocket.patch_action(id=id,json={"total_count": action_count})


In [7]:
def unknown_function():
    old_text = "’"
    old_actions = pocket.get_actions(params={'filter': f'action~"{old_text}"'},full=False)
    new_text = "'"
    new_actions = pocket.get_actions({'filter': f'action~"{new_text}"'},full=False)

    for key in list(old_actions.keys()):
        old_id = old_actions[key]
        new_id = new_actions[key.replace(old_text, new_text)]
        days = pocket.get_days(params={'filter': f'action="{old_id}"'}, full=True).json()['items']
        for day in days:
            pocket.patch_day(id=day['id'],json={'action': new_id})

In [8]:
level_one = pocket.get_levels('level_one',params={},full=False)

In [9]:
breakdown = {}
for cat in list(level_one.keys()):
    breakdown[cat] = sum(map(lambda x: x['total_count'], get_all(pocket.get_actions,params={'filter': f'level_one.classification="{cat}"'})))

In [11]:
breakdown

{'Sleep': 5119, 'Work': 5894, 'Life': 3903, 'Food': 770, 'Unknown': 1782}

In [13]:
# Helper function
def count_to_time(count: int):
    time = f"{int(count // (4))} hours, {int(((count % (4))/(4))*(60))} minutes"
    return time

In [18]:
# time_spent(data) will return a dictionary with the values mapped from counts to total time spent
def time_spent(data: dict):
    return {k: count_to_time(v) for k, v in data.items()} 

In [19]:
def average_day(data: dict):
    total = sum(data.values())
    fractions = {k: (v/total)*96 for k, v in data.items()}
    return time_spent(fractions)

In [20]:
time_spent(breakdown)

{'Sleep': '1279 hours, 45 minutes',
 'Work': '1473 hours, 30 minutes',
 'Life': '975 hours, 45 minutes',
 'Food': '192 hours, 30 minutes',
 'Unknown': '445 hours, 30 minutes'}

In [21]:
average_day(breakdown)

{'Sleep': '7 hours, 1 minutes',
 'Work': '8 hours, 5 minutes',
 'Life': '5 hours, 21 minutes',
 'Food': '1 hours, 3 minutes',
 'Unknown': '2 hours, 26 minutes'}

In [22]:
import datetime

In [56]:
# select_time_range(start,end,filter,expand) will return a list of days betweem `start` and 
# `end` (inclusive start, exclusive end). `filter` is a list of filter parameters for the GET
# request.
def select_time_range(start: datetime.datetime, end: datetime.datetime, filter: list, expand: str):

    start_UTC = pocket.to_UTC(start, text=True)
    end_UTC = pocket.to_UTC(end, text=True)

    params = {}
    # Adding filter parameters
    filters = [f'(date>="{start_UTC}")', f'(date<"{end_UTC}")'] + filter
    params.update({'filter': ' && '.join(filters)})
    # Adding expand parameters
    params.update({'expand': expand})

    days = get_all(pocket.get_days, params=params)
    return days

In [60]:
# level_two_by_week(start,end,classification) will return a list of tuples. These tuples
# are the start dates, end dates, and counts of `classification` as level two over the 7-day 
# periods between `start` and `end`. 7-day periods are made to start on Monday.
def level_two_by_week(start:datetime.datetime,end:datetime.datetime,classification:str):

    counts = []
    check_date = start

    if (start.weekday() != 0):
        weekday = start.weekday()
        check_date = start + datetime.timedelta(days=weekday)
        count = select_time_range(
            start=start,
            end=check_date,
            filter=[f'(action.level_two.classification="{classification}")'],
            expand='')
        counts.append((start,check_date,len(count)))

    while(check_date < end):

        count = select_time_range(
            start=check_date,
            end=check_date + datetime.timedelta(days=7),
            filter=[f'(action.level_two.classification="{classification}")'],
            expand='')
        counts.append((check_date,check_date + datetime.timedelta(days=7),len(count)))
        check_date = check_date + datetime.timedelta(days=7)

    check_date = check_date - datetime.timedelta(days=7)
    count = select_time_range(
            start=check_date,
            end=end,
            filter=[f'(action.level_two.classification="{classification}")'],
            expand='')
    counts.append((check_date,end,len(count)))

    return counts

In [61]:
start_date = datetime.datetime(year=2021,month=8,day=1)
end_date = datetime.datetime(year=2021,month=12,day=30)

start = pocket.create_eastern_datetime(start_date, '06:00')
end = pocket.create_eastern_datetime(end_date, '21:00')

len(select_time_range(
            start=start,
            end=end,
            filter=[f'(action.level_two.classification="Pi Kappa Alpha")'],
            expand=''))

723

In [62]:
level_two_by_week(start=start,end=end,classification='GOVT 3112')

[(datetime.datetime(2021, 8, 1, 6, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000), 'EDT')),
  datetime.datetime(2021, 8, 7, 6, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000), 'EDT')),
  0),
 (datetime.datetime(2021, 8, 7, 6, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000), 'EDT')),
  datetime.datetime(2021, 8, 14, 6, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000), 'EDT')),
  0),
 (datetime.datetime(2021, 8, 14, 6, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000), 'EDT')),
  datetime.datetime(2021, 8, 21, 6, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000), 'EDT')),
  0),
 (datetime.datetime(2021, 8, 21, 6, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000), 'EDT')),
  datetime.datetime(2021, 8, 28, 6, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000), 'EDT')),
  0),
 (datetime.datetime(2021, 8, 28, 6, 0, tzin